In [1]:
import site
import pickle as pkl
import h5py as hp
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import copy
import seaborn
DRAFTPATH = '/home/cosinga/hicolor_draft/figures/%s'
def siteFG():
    FGPATH = '/home/cosinga/figrid/'
    site.addsitedir(FGPATH)
    return


def trgba(attr, value, alpha = 1.0):
    return mpl.colors.to_rgba(ds.attr_args[attr][value]['color'], alpha)

def rgba(color, alpha):
    return mpl.colors.to_rgba(color, alpha)
siteFG()
from figrid.data_sort import DataSort
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rcParams['font.family'] = 'serif'

def makeBlueRedRatio(datalist):
    print("making pk ratios")
    from figrid.data_container import DataContainer
    ip = {'color':'blue', 'color_cut':'0.60', 'path':['galbt', 'HIbt'], 'post_process':'no key found'}
    blues = datalist.getMatching(ip)
    ratiolist = []
    count = 0
    pnt_threshold = 0.05
    for dc in blues:
        mattr = copy.deepcopy(dc.attrs)
        rmattr = []
        for k in mattr:
            if 'runtime' in k or 'color' == k:
                rmattr.append(k)
        for rm in rmattr:
            del mattr[rm]
        mattr['color'] = 'red'
        mattr['post_process'] = 'no key found'
        reds = datalist.getMatching(mattr)
        if len(reds) > 1:
            print('%d corresponding reds for a blue:'%len(reds))
            print("BLUE ATTRS:")
            print(dc.attrs)
            print("RED ATTRS:")
            for r in reds:
                print(r.attrs)
        if len(reds) == 0:
            print('no reds found')
            print(dc.attrs)
        else:
            reds = reds[0]
            data = [dc.data[0], reds.data[1]/dc.data[1]]
            ratio = DataContainer(data)
            mattr['color'] = 'ratio' 
            ratio.update(mattr)
            ratiolist.append(ratio)
        if count / len(blues) >= pnt_threshold:
            print("calculated bias for %.2f"%pnt_threshold)
            pnt_threshold += 0.05
    return ratiolist

def makeBlueRedCCRatio(datalist):
    print("calculating CC ratios")
    from figrid.data_container import DataContainer
    ip = {'color':'blue', 'color_cut':'0.60', 'path':['HIbt', 'galbt'], 'post_process':'corr_coef'}
    blues = datalist.getMatching(ip)
    ratiolist = []
    count = 0
    pnt_threshold = 0.05
    for dc in blues:
        mattr = copy.deepcopy(dc.attrs)
        rmattr = []
        for k in mattr:
            if 'runtime' in k or 'color' == k:
                rmattr.append(k)
        for rm in rmattr:
            del mattr[rm]
        mattr['color'] = 'red'
        reds = datalist.getMatching(mattr)
        if len(reds) > 1:
            print('%d corresponding reds for a blue:'%len(reds))
            print("BLUE ATTRS:")
            print(dc.attrs)
            print("RED ATTRS:")
            for r in reds:
                print(r.attrs)
        if len(reds) == 0:
            print('no reds found')
            print(dc.attrs)
        else:
            reds = reds[0]
            data = [dc.data[0], reds.data[1]/dc.data[1]]
            ratio = DataContainer(data)
            mattr['color'] = 'ratio'
            mattr['post_process'] = 'corr_ratio'
            ratio.update(mattr)
            ratiolist.append(ratio)
        if count / len(blues) >= pnt_threshold:
            print("calculated bias for %.2f"%pnt_threshold)
            pnt_threshold += 0.05
    return ratiolist

SPECS = {}
SPECS['all'] = ['box', 'simname', 'grid_resolution', 'snapshot', 'axis', 'path', 'mas', 'space']
SPECS['hisubhalo'] = ['model', 'HI_res', 'censat']
SPECS['galaxy'] = ['color', 'gal_species', 'gal_res', 'color_cut', 'censat']
SPECS['ptl'] = ['ptl_species']
SPECS['hiptl'] = ['model']
SPECS['vn'] = []
def makeBias(datalist, path):
    print("making biases")
    from figrid.data_container import DataContainer
    biaslist = []
    
    crosses = datalist.getMatching({'is_auto' : False, 'path':path})
    cx_count = 0
    pnt_threshold = 0.05
    for cx in crosses:
        fns = cx.get('fieldname').split('_')
        if 'dust' in fns:
            continue
        autos = []
        for f in range(len(fns)):
            attrlist = copy.deepcopy(SPECS['all'])
            attrlist.extend(SPECS[fns[f]])
            attr = {'is_auto':True, 'fieldname':fns[f]}
            for at in attrlist:
                attr[at] = cx.get(at)
            
            if 'censat' in attr:
                if '_' in attr['censat']:
                    attr['censat'] = attr['censat'].split('_')[f]
            matches = datalist.getMatching(attr)
            if len(matches)>1:
                print('too many matches %d'%len(matches))
                for m in matches:
                    print(m.attrs)
            elif len(matches) == 0:
                print('no matches found')
                print(attr)
                print(cx.attrs)
            else:
                auto = matches[0]
#                 data = [cx.data[0], cx.data[1] / auto.data[1]]
#                 dc = DataContainer(data)
#                 dc.update(copy.deepcopy(cx.attrs))
#                 dc.add('post_process', 'theory_bias')
#                 dc.add('numerator', cx.get('fieldname'))
#                 dc.add('denominator', auto.get('fieldname'))
#                 biaslist.append(dc)
                autos.append(auto)
        if len(autos) == 2:
#             for a in range(len(autos)):
#                 data = [autos[a].data[0], np.sqrt(autos[a].data[1] / autos[(a+1)%2].data[1])]
#                 dc = DataContainer(data)
#                 dc.update(copy.deepcopy(cx.attrs))
#                 dc.add('post_process', 'obs_bias')
#                 dc.add('numerator', autos[a].get('fieldname'))
#                 dc.add('denominator', autos[(a+1)%2].get('fieldname'))
#                 biaslist.append(dc)

            data = [autos[0].data[0], cx.data[1] / np.sqrt(autos[0].data[1] * autos[1].data[1])]
            dc = DataContainer(data)
            dc.update(copy.deepcopy(cx.attrs))
            dc.add('post_process', 'corr_coef')
            biaslist.append(dc)
        cx_count += 1
        if cx_count / len(crosses) >= pnt_threshold:
            print("calculated bias for %.2f"%pnt_threshold)
            pnt_threshold += 0.05
        del attr
    return biaslist

In [2]:
ds = pkl.load(open('/home/cosinga/hcolor/fig_md_analysis/bias_results.pkl', 'rb'))
galbt_coef = makeBias(ds, 'galbt')
hibt_coef = makeBias(ds, 'HIbt')
ratio = makeBlueRedRatio(ds)
ds.loadHdf5('/home/cosinga/population_hists.hdf5')
ds.extend(galbt_coef); ds.extend(hibt_coef); ds.extend(ratio)
corr_ratio = makeBlueRedCCRatio(ds)
ds.extend(corr_ratio)
pkl.dump(ds, open('binth_coef.pkl', 'wb'), pkl.HIGHEST_PROTOCOL)

making biases
calculated bias for 0.05
calculated bias for 0.10
calculated bias for 0.15
calculated bias for 0.20
calculated bias for 0.25
calculated bias for 0.30
calculated bias for 0.35
calculated bias for 0.40
calculated bias for 0.45
calculated bias for 0.50
calculated bias for 0.55
calculated bias for 0.60
calculated bias for 0.65
calculated bias for 0.70
calculated bias for 0.75
calculated bias for 0.80
calculated bias for 0.85
calculated bias for 0.90
calculated bias for 0.95
making biases
calculated bias for 0.05
calculated bias for 0.10
calculated bias for 0.15
calculated bias for 0.20
calculated bias for 0.25
calculated bias for 0.30
calculated bias for 0.35
calculated bias for 0.40
calculated bias for 0.45
no matches found
{'is_auto': True, 'fieldname': 'galaxy', 'box': 75.0, 'simname': 'tng100', 'grid_resolution': 800, 'snapshot': 99, 'axis': 0, 'path': 'HIbt', 'mas': 'CICW', 'space': 'real', 'color': 'red_blue', 'gal_species': 'stmass', 'gal_res': 'diemer', 'color_cut': '